In [1]:
import pandas as pd
import os
import gc

In [2]:
# Read in the bulk individual contribution files from FEC.gov and combine

# Directory containing the text files
directory = './indiv24/by_date'

chunk_size = 200000  # Number of rows per chunk, reduce if kernel dies (memory overload)
column_names = [
    'ID', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Name', 'City', 'State', 'Zip',
    'Employer', 'Occupation', 'Date', 'Amount', 'OtherID', 'Field16', 'Field17', 'Field18', 'Field19', 'Field20'
]

dfs = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path, delimiter='|', names=column_names, header=None, low_memory=False)
        dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
pd.set_option('display.max_columns', None)
combined_df.head()

,ID,Field1,Field2,Field3,Field4,Field5,Field6,Name,City,State,...,Employer,Occupation,Date,Amount,OtherID,Field16,Field17,Field18,Field19,Field20
0,C00079681,N,M6,P,202306129581782820,15,IND,"GODLEWSKI, KIM",FENTON,MI,...,IPS EQUIPMENT,VICE PRESIDENT,5282023.0,100,NaN,2425123,1707004,NaN,NaN,4061220231749193699
1,C00409730,N,M6,P,202306129582118675,15E,IND,"ROBERTS, PHILIP",SEATTLE,WA,...,"RYAN, SWANSON & CLEVELAND, PLLC",ATTORNEY,5282023.0,500,C00699470,15444361,1707040,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4061320231749244130
2,C00409730,N,M6,P,202306129582118670,15E,IND,"GRUNDSTEIN, LEON",MERCER ISLAND,WA,...,"GENCARE, INC","SENIOR LIVING OWNER, DEVELOPER, OPERAT",5282023.0,500,C00699470,15444363,1707040,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4061320231749244117
3,C00409730,N,M6,P,202306129582118676,15E,IND,"SCHOCKEN, JOSEPH L.",SEATTLE,WA,...,BROADMARK CAPITAL,PRESIDENT,5282023.0,5000,C00699470,15444362,1707040,NaN,* EARMARKED CONTRIBUTION: SEE BELOW,4061320231749244134
4,C00063164,N,M6,P,202306139582145211,15,IND,"MCMILLAN, S TODD",COLORADO SPRINGS,CO,...,MCDONALD'S,LICENSEE,5282023.0,200,NaN,17411206,1707152,NaN,NaN,4061320231749248513


In [4]:
# Create a primary key column by combining Name and Zip
combined_df['PrimaryKey'] = combined_df['Name'] + '_' + combined_df['Zip'].astype(str)

# Group by the primary key and aggregate
final_df = combined_df.groupby('PrimaryKey').agg(
    RecordCount=('PrimaryKey', 'size'),
    TotalAmount=('Amount', 'sum')
).reset_index()
final_df = final_df.sort_values(by='RecordCount', ascending=False)
final_df.head(10)

,PrimaryKey,RecordCount,TotalAmount
1682585,"MAKOWSKI, BRUCE_480171279",20092,60654
1311445,"JAHNS, RICHARD_760405951",18705,360303
867301,"FORD, BEVERLY_300673614",16791,155112
810454,"FARR, GERALD_786665129",15304,203311
1427098,"KESKITALO, CANDACE_55384",12724,71559
2781986,"URBANOWICZ, WENDY_98668",11912,64469
1682583,"MAKOWSKI, BRUCE_48017",11038,31870
1049787,"GRISWOLD, EDSON_80231",10642,141357
1630983,"LOONEY, GEORGE_300402707",10587,26656
2567262,"SOLORZANO, DIANA_92056",9963,8914


In [5]:
# Issue: Some entries include middle name or title, or change use a zip+4 code in place of a zip code
filtered_df = final_df.loc[final_df['PrimaryKey'].str.startswith('MAKOWSKI, BRUCE')]
print(filtered_df)

                                   PrimaryKey  RecordCount  TotalAmount
1682585             MAKOWSKI, BRUCE_480171279        20092        60654
1682583                 MAKOWSKI, BRUCE_48017        11038        31870
1682579      MAKOWSKI, BRUCE EDWARD_480171279         2146         6945
1682578  MAKOWSKI, BRUCE EDWARD MR._480171279          342         3004
1682584             MAKOWSKI, BRUCE_480171227          230         1028
1682580         MAKOWSKI, BRUCE MR._480171279           94          583
1682582                 MAKOWSKI, BRUCE_47985            5            4
1682581              MAKOWSKI, BRUCE MR_48017            4            6


In [26]:
# Solution: Only use first 5 digits of Zip and first two words of name (first and last)
# Remove previous result dataframes from memory
if 'final_df' in locals():
    del final_df
if 'filtered_df' in locals():
    del filtered_df
gc.collect()

combined_df['Name'] = combined_df['Name'].fillna('').astype(str)
combined_df['PrimaryName'] = combined_df['Name'].apply(lambda x: ' '.join(x.split()[:2]))
combined_df['PrimaryZip'] = combined_df['Zip'].astype(str).str[:5]
combined_df['PrimaryKey'] = combined_df['PrimaryName'] + '_' + combined_df['PrimaryZip']
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%m%d%Y')
corrected_final_df = combined_df.groupby(['PrimaryKey']).agg(
    RecordCount=('PrimaryKey', 'size'),
    TotalAmount=('Amount', 'sum')
).reset_index()

# Dataframe without zip+4 specificity and without middle names or titles
corrected_final_df = corrected_final_df.sort_values(by='RecordCount', ascending=False)
pd.set_option('display.max_rows', None)
corrected_final_df.head(10)

,PrimaryKey,RecordCount,TotalAmount
1182436,"MAKOWSKI, BRUCE_48017",33946,104090
610786,"FORD, BEVERLY_30067",23704,212363
921807,"JAHNS, RICHARD_76040",23055,462964
1003036,"KESKITALO, CANDACE_55384",20477,134036
1957061,"URBANOWICZ, WENDY_98668",18657,159470
1533107,"PRESLEY, JUANITA_83642",16211,249974
1145859,"LOONEY, GEORGE_30040",15798,38273
738416,"GRISWOLD, EDSON_80231",15630,260486
571009,"FARR, GERALD_78666",15469,206141
1389896,"NGUYEN, KATHERINE_21740",14719,12630


In [27]:
# Let's look at the donors who gave the most amount of times in a single day
if 'corrected_final_df' in locals():
    del corrected_final_df
gc.collect()
daily_corrected_final_df = combined_df.groupby(['PrimaryKey', 'Date']).agg(
    RecordCount=('PrimaryKey', 'size'),
    TotalAmount=('Amount', 'sum')
).reset_index()
daily_corrected_final_df = daily_corrected_final_df.sort_values(by='RecordCount', ascending=False)

In [28]:
daily_corrected_final_df.head(10)

,PrimaryKey,Date,RecordCount,TotalAmount
15280446,"ZARCONE, ANNAMAE_11004",2023-09-06,4441,12876
9063512,"MCKEE, LAURENCE_29485",2024-03-26,3218,24474
1934189,"CAFFREY, DOROTHY_60126",2023-11-01,2824,22465
8254480,"LOONEY, GEORGE_30040",2024-02-23,2724,8519
7351897,"KIPPUR, SUSAN_10023",2023-12-10,2486,26240
5865322,"HEATON, VONNIE_30101",2024-10-09,2394,10150
10916180,"POHOST, GERALD_90005",2024-02-27,2178,8484
6663259,"JAHNS, RICHARD_76040",2023-10-06,2132,39759
6663263,"JAHNS, RICHARD_76040",2023-10-10,2082,26698
4045746,"EVIDON, EVE_55436",2023-11-14,1948,13742


In [29]:
# There were 4,441 transactions attributed to Annamae Zarcone on 2023-09-06 alone
zarcone_96_df =  combined_df[(combined_df['PrimaryKey'] == "ZARCONE, ANNAMAE_11004") & (combined_df['Date'] == "2023-09-06")]
zarcone_96_df = zarcone_96_df[['Name', 'Date', 'Amount']]

zarcone_96_df.head(10)

,Name,Date,Amount
9513647,"ZARCONE, ANNAMAE",2023-09-06,0
9513648,"ZARCONE, ANNAMAE",2023-09-06,0
9513649,"ZARCONE, ANNAMAE",2023-09-06,0
9513650,"ZARCONE, ANNAMAE",2023-09-06,0
9513651,"ZARCONE, ANNAMAE",2023-09-06,0
9513652,"ZARCONE, ANNAMAE",2023-09-06,0
9513653,"ZARCONE, ANNAMAE",2023-09-06,6
9513654,"ZARCONE, ANNAMAE",2023-09-06,1
9513655,"ZARCONE, ANNAMAE",2023-09-06,1
9513656,"ZARCONE, ANNAMAE",2023-09-06,0


In [33]:
# Many of the transactions are zero or negative, apparently in reference to chargebacks or adjustments to previous donations
# The negative transactions do not positively contribute to the TotalAmount sum
# Let's count the negative and zero valued transactions
negative_count = (zarcone_96_df['Amount'] < 0).sum()
zero_count = (zarcone_96_df['Amount'] == 0).sum()

# Print the counts
print(f'Number of negative results: {negative_count}')
print(f'Number of zero results: {zero_count}')
pos_count = len(zarcone_96_df) - zero_count - negative_count
print(f'Number of positive results: {pos_count}')
seconds_in_day = 24 * 60 * 60
print(f'Average of {round(seconds_in_day / len(zarcone_96_df), 3)}s between each transaction of any type')
print(f'Average of {round(seconds_in_day / pos_count, 3)}s between each positive transaction')

Number of negative results: 681
Number of zero results: 884
Number of positive results: 2876
Average of 19.455s between each transaction of any type
Average of 30.042s between each positive transaction
